### 데이터프레임의 결합 
- 유니언 결합 
    - 단순하게 행을 결합하거나 열을 결합하는 방식
    - pandas에 내장된 concat() 함수를 이용
        - axis 매개변수
            - 0(rows) : 행을 결합 (기본값)
            - 1(columns) : 열을 결합 
        - ignore_index 매개변수
            - False(기본값) : 결합되는 인덱스들을 그대로 보존
            - True : 결합되는 인덱스를 초기화
    - 다수의 데이터프레임을 한번에 결합이 가능
- 조인 결합 
    - 특정한 조건에 맞게 열을 결합 
    - 두개의 데이터프레임만 결합 가능
    - pandas에 내장된 merge() 함수를 이용
        - merge(데이터프레임명, 데이터프레임명)
        - on 매개변수 
            - 조건식(두 개의 데이터프레임이 공통적으로 가지고 있는 컬럼의 이름)
            - 컬럼의 이름이 다른 경우 
                - left_on, right_on 매개변수를 이용해서 결합이 가능
        - how 매개변수 
            - 두 개의 데이터프레임을 어떠한 방식으로 결합 
            - left : 왼쪽의 데이터프레임을 기준으로 열 결합 
            - right : 오른쪽의 데이터프레임을 기준으로 열 결합
            - inner : 두개의 데이터프레임이 공통적으로 가지고 있는 데이터들을 기준으로 결합 
            - outer : 두개의 데이터프레임을 합집합의 형태로 결합 

In [1]:
import pandas as pd

In [ ]:
data = {
    'name' : ['test', 'test2', 'test3'], 
    'age' : [20, 30, 40]
}
df = pd.DataFrame(data)
df

In [ ]:
data2 = {
    'name' : ['test4', 'test5'], 
    'loc' : ['seoul', 'busan']
}
df2 = pd.DataFrame(data2)
df2

In [ ]:
# 단순한 행 결합 
pd.concat([df, df2], axis=0)
pd.concat([df, df2], ignore_index=True)
pd.concat([df, df2]).reset_index(drop=True)

In [ ]:
# 단순한 열 결합 
pd.concat([df, df2], axis=1)
pd.concat([df, df2], axis=1, ignore_index=True)

In [ ]:
data3 = {
    'loc' : ['seoul', 'busan'], 
    'code' : ['11', '51']
}
df3 = pd.DataFrame(data3, index=[5, 6])
df3

In [ ]:
pd.concat([df, df2, df3], axis=0)

In [ ]:
pd.concat([df, df2, df3], axis=1)

In [ ]:
data4 = {
    'id' : ['test', 'test2', 'test3'],
    'pass' : ['1234', '1111', '0000']
}
id_table = pd.DataFrame(data4)
id_table

In [14]:
data5 = {
    'id' : ['test', 'test2'], 
    'item' : ['A', 'C']
}
id_item = pd.DataFrame(data5)
id_item

,id,item
0,test,A
1,test2,C


In [20]:
merge_data = pd.merge(
    id_table, id_item,
    on = 'id', 
    how = 'outer'
)

In [21]:
data6 = {
    'item_name' : ['A', 'B'], 
    'price' : [100, 200]
}
item_table = pd.DataFrame(data6)
item_table

,item_name,price
0,A,100
1,B,200


In [ ]:
pd.merge(
    merge_data, item_table, 
    left_on = 'item', 
    right_on = 'item_name', 
    how='outer'
)

In [28]:
merge_data.loc[2, 'item'] = 'A'

In [29]:
merge_data

,id,pass,item
0,test,1234,A
1,test2,1111,C
2,test3,0000,A


In [32]:
# item_table에 item_name이라는 컬럼의 이름을 item으로 변경
item_table.rename(
    columns = {
        'item_name' : 'item'
    }, inplace=True
)

In [33]:
pd.merge(
    merge_data, item_table, 
    on = 'item', 
    how = 'inner'
)

,id,pass,item,price
0,test,1234,A,100
1,test3,0000,A,100


- 특정 경로에 있는 파일의 목록을 가지고 와서 반복문을 이용하여 모든 데이터를 하나의 데이터프레임으로 결합
    1. 특정 경로에 있는 파일 목록 로드 
    2. 반복문을 이용하여 데이터를 하나씩 로드 
    3. 단순한 행 결합으로 데이터프레임을 결합 

In [2]:
import os 
from glob import glob

In [ ]:
# os 라이브러리를 이용한 파일 목록 불러오기 
os.listdir('../csv/2017')

In [43]:
# 특정 경로를 변수에 저장 
file_path = '../csv/2017/'
file_list = os.listdir(file_path)

In [ ]:
file_list

In [45]:
file_path + file_list[0]

'../csv/2017/201701_expense_list.csv'

In [ ]:
pd.read_csv(file_path + file_list[0])

In [50]:
# 2017년의 1-12월까지의 데이터를 하나의 데이터프레임으로 생성 

# 빈 데이터프레임을 생성 
df_2017 = pd.DataFrame()

# file_list를 기준으로 반복 실행하는 반복문을 생성 
for file in file_list:
    # file변수에 대입되는 값은 ? 파일의 이름
    # print(file)
    df = pd.read_csv(file_path + file)
    # df_2017에 df을 대입(단순한 행 결합을 하여 df_2017에 넣어준다.)
    df_2017 = pd.concat([df_2017, df])

In [51]:
df_2017.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70132 entries, 0 to 7285
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   nid                70132 non-null  int64  
 1   title              70132 non-null  object 
 2   url                70132 non-null  object 
 3   dept_nm_lvl_1      70132 non-null  object 
 4   dept_nm_lvl_2      70074 non-null  object 
 5   dept_nm_lvl_3      61262 non-null  object 
 6   dept_nm_lvl_4      17939 non-null  object 
 7   dept_nm_lvl_5      3474 non-null   object 
 8   exec_yr            70132 non-null  int64  
 9   exec_month         70132 non-null  int64  
 10  expense_budget     3108 non-null   float64
 11  expense_execution  2805 non-null   float64
 12  category           1259 non-null   object 
 13  dept_nm_full       70053 non-null  object 
 14  exec_dt            70132 non-null  object 
 15  exec_loc           69360 non-null  object 
 16  exec_purpose       70110 non

In [54]:
# glob 라이브러리 
glob("../csv/2021/*.csv")

['../csv/2021\\202101_expense_list.csv',
 '../csv/2021\\202102_expense_list.csv',
 '../csv/2021\\202103_expense_list.csv',
 '../csv/2021\\202104_expense_list.csv',
 '../csv/2021\\202105_expense_list.csv',
 '../csv/2021\\202106_expense_list.csv',
 '../csv/2021\\202107_expense_list.csv',
 '../csv/2021\\202108_expense_list.csv',
 '../csv/2021\\202109_expense_list.csv',
 '../csv/2021\\202110_expense_list.csv',
 '../csv/2021\\202111_expense_list.csv',
 '../csv/2021\\202112_expense_list.csv']

In [3]:
# 지정된 경로에 파일들을 하나의 데이터프레임으로 결합하는 함수 
def data_load(_path, _end = 'csv'):
    # _path : 파일의 경로
    # _end : 파일의 확장자

    # 유저가 입력한 경로를 이용하여 파일의 목록을 로드 
    file_list = os.listdir(_path)
    # 혹시 유저가 경로의 마지막에 '/'가 없는 경우를 대비하여 마지막에 '/' 추가 
    file_path = _path + '/'

    # 비어있는 데이터프레임을 생성 
    result = pd.DataFrame()

    for file in file_list:
        # file -> 파일의 이름 
        # _end의 값에 따라 다른 함수를 이용
        if _end == 'csv':
            df = pd.read_csv(file_path + file)
        elif _end == 'json':
            df = pd.read_json(file_path + file)
        elif (_end == 'xlsx') | (_end =='xls'):
            df = pd.read_excel(file_path + file)
        elif _end == 'xml':
            df = pd.read_xml(file_path + file)
        else:
            return "지원하지 않은 확장자입니다."
        # 비어있는 데이터프레임에 단순 행 결합 
        result = pd.concat([result, df])
    print("데이터프레임 모두 결합 완료 크기는 ", len(result))
    return result

In [ ]:
data_load('../csv/2017')

In [ ]:
data_load("../csv/2019/", 'json')

In [5]:
data_load("../csv/2020/", 'xlsx')

데이터프레임 모두 결합 완료 크기는  70862


,nid,title,url,dept_nm_lvl_1,dept_nm_lvl_2,dept_nm_lvl_3,dept_nm_lvl_4,dept_nm_lvl_5,exec_yr,exec_month,...,expense_execution,category,dept_nm_full,exec_dt,exec_loc,exec_purpose,target_nm,payment_method,exec_amount,bimok
0,21789676,2020년 1월 사업소 서울식물원 시설운영과 업무추진비 - 부서운영,http://opengov.seoul.go.kr/public/21789676,사업소,서울식물원,시설운영과,NaN,NaN,2020,1,...,NaN,NaN,푸른도시국 서울식물원 시설운영과,2020-01-22 10:25,코코호도마곡나루역점(강서구 마곡중앙5로 6),부서 전출직원 간담회 비용,이형선 외 3인,카드,30000,NaN
1,21762080,2020년 1월 사업소 상수도사업본부 남부수도사업소 행정지원과 업무추진비 - 부서운영,http://opengov.seoul.go.kr/public/21762080,사업소,상수도사업본부,남부수도사업소,행정지원과,NaN,2020,1,...,NaN,NaN,남부수도사업소 행정지원과,2020-01-29 08:55,"㈜한국문화진흥(강남구 강남대로 636, 두원빌딩2층)",2020년도 1월 생일자 격려상품 구매,1월 직원 생일자 12명,카드,216720,NaN
2,21762080,2020년 1월 사업소 상수도사업본부 남부수도사업소 행정지원과 업무추진비 - 부서운영,http://opengov.seoul.go.kr/public/21762080,사업소,상수도사업본부,남부수도사업소,행정지원과,NaN,2020,1,...,NaN,NaN,남부수도사업소 행정지원과,2020-01-28 12:05,스시사도(영등포구 여의대방로23길 20),2020년 상반기 학교아리수 음수대 감면 간담회 경비지급,"박영준,이상봉,이재욱,이대영,김춘근,김근태,박화준,박병국,조승용,전윤수",카드,124000,NaN
3,21762080,2020년 1월 사업소 상수도사업본부 남부수도사업소 행정지원과 업무추진비 - 부서운영,http://opengov.seoul.go.kr/public/21762080,사업소,상수도사업본부,남부수도사업소,행정지원과,NaN,2020,1,...,NaN,NaN,남부수도사업소 행정지원과,2020-01-23 11:59,올갱이나라(영등포구 대방천로 234),2020년 설연휴기간 급수대책 및 수질관리 간담회 경비지급,"김지형,신민철,서영식,최신,이혜숙,여종순",카드,48000,NaN
4,21762080,2020년 1월 사업소 상수도사업본부 남부수도사업소 행정지원과 업무추진비 - 부서운영,http://opengov.seoul.go.kr/public/21762080,사업소,상수도사업본부,남부수도사업소,행정지원과,NaN,2020,1,...,NaN,NaN,남부수도사업소 행정지원과,2020-01-23 14:45,두드림유통(동작구 보라매로19길 8),2020년 민원실 직원 격려 물품구매 경비지급,해당사항없음,카드,63310,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7810,22228979,2020년 12월 사업소 서울시립대학교 자유융합대학 업무추진비 - 전체,http://opengov.seoul.go.kr/public/22228979,사업소,서울시립대학교,자유융합대학,NaN,NaN,2020,12,...,NaN,NaN,자유융합대학,2020-12-29 14:28,죠샌드위치(동대문구 서울시립대로 147 103호),교양교육부 강사 채용 다과 및 음료 구입,학장 외 2명,카드,36000,NaN
7811,22228979,2020년 12월 사업소 서울시립대학교 자유융합대학 업무추진비 - 전체,http://opengov.seoul.go.kr/public/22228979,사업소,서울시립대학교,자유융합대학,NaN,NaN,2020,12,...,NaN,NaN,자유융합대학,2020-12-24 11:32,본도시락(동대문구전농로209),자유융합대학 부서회의 간담회비(회의종료 후 개별지급),학장 외 5명,카드,89400,NaN
7812,22228979,2020년 12월 사업소 서울시립대학교 자유융합대학 업무추진비 - 전체,http://opengov.seoul.go.kr/public/22228979,사업소,서울시립대학교,자유융합대학,NaN,NaN,2020,12,...,NaN,NaN,자유융합대학,2020-12-04 13:45,꽃갈피(동대문구 망우로8가길),융합전공학부 관서업무추진비(교과과정 개편 관련 간담회),학장 외 2명,카드,11000,NaN
7813,22241147,2020년 12월 사업소 물재생센터 난지물재생센터 시설보수과 업무추진비 - 부서운영,http://opengov.seoul.go.kr/public/22241147,사업소,물재생센터,난지물재생센터,시설보수과,NaN,2020,12,...,NaN,NaN,물순환안전국 난지물재생센터 시설보수과,2020-12-28 15:18,강서농협하나로마트,탕비용품 구매,조선행과장외 22명,카드,687700,NaN
